In [1]:
from argparse import Namespace

from classifier import *
from dataset import *
from vectorizer import *

import torch

/home/nik/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Matplotlib created a temporary config/cache directory at /tmp/matplotlib-_q2ftbrl because the default path (/home/nik/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


## Namespace
Here we use a namespace to aggregate all runtime parameters and easily adjust them later on

In [2]:
def use_cuda():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

args = Namespace (
    in_features=None,
    out_units=1,
    dataset=None,
    optimizer="Adam",
    criterion="bce_logits",
    batch_size=128,
    learning_rate=0.001,
    num_epochs=5,
    device=use_cuda(),
    embed_dim=300,
    freeze_embedding=False,
    filter_sizes=[3, 4, 5],
    num_filters=[100, 100, 100],
    pretrained_embedding=None, 
    hidden_size=2, # number of features in hidden state
    num_layers=1, # number of stacked lstm layers,
    save_state_path="../checkpoints",
    save_best_path="../best_models",
    name=""
)

Load the tweets products dataset

In [3]:
args.dataset = TextDataset.load_dataset_and_make_vectorizer("../data/tweets_products.csv")
args.in_features = args.dataset.vectorizer.max_padding
args.name = "BOW-Classifier.pth"

[nltk_data] Downloading package stopwords to /home/nik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Bag of Words Classifier

Initialize a simple neural network which utilizes a bag of words approach to representing the encoded text. 

In [4]:
bow = BOWClassifier(args=args)
bow.setup()
bow.fit()

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
bow.plot_logs("Accuracy", ["Train Accuracy", "Validation Accuracy"])

In [ ]:
bow.plot_logs("Loss", ["Train Loss", "Validation Loss"])

In [ ]:
bow.plot_logs("F1", ["Train F1-Score", "Validation F1-Score"])

Evaluate the the bag of words classifier on the tweets products dataset.

In [ ]:
bow.load_state()
loss, acc, f1 = bow.eval_net(mode='test')
print(loss)
print(acc)
print(f1)

Now we load the IMDB dataset and test the above model on that. 
We observed a significant drop in the performance of the model as it can be seen from the output of the current cell.

In [ ]:
args.dataset = TextDataset.load_dataset_and_make_vectorizer("../data/IMDB-dataset.csv")
loss, acc, f1 = bow.eval_net(mode='test')
print(loss)
print(acc)
print(f1)

In the next experiments we would like to evaluate the performance of the convolutional neural network on the previous scenario.

In [ ]:
args.dataset = TextDataset.load_dataset_and_make_vectorizer("../data/tweets_products.csv", vectorizer_mode="padded")
args.pretrained_embedding = args.dataset.vectorizer.load_pretrained_embed("../data/crawl-300d-2M.vec")
args.name = "CNN-Classifier.pth"

### Convolutional Neural Network

Setup a CNN classifier, train and validate the network on the tweets products dataset.

In [ ]:
cnn_classifier = CNNClassifier(args=args)
cnn_classifier.setup()
cnn_classifier.fit()

In [ ]:
cnn_classifier.plot_logs("Accuracy", ["Train Accuracy", "Validation Accuracy"])

In [ ]:
cnn_classifier.plot_logs("Loss", ["Train Loss", "Validation Loss"])

In [ ]:
cnn_classifier.plot_logs("F1", ["Train F1-Score", "Validation F1-Score"])

Test CNN on the tweets test set

In [ ]:
cnn_classifier.load_state()
loss, acc, f1 = cnn_classifier.eval_net(mode='test')
print(loss)
print(acc)
print(f1)

Evaluate CNN's performance on the IMDB dataset. Again there is decrease on the new test set, although not as significant as in the previous case.

In [ ]:
args.dataset = TextDataset.load_dataset_and_make_vectorizer("../data/IMDB-dataset.csv", vectorizer_mode="padded")
embedding = args.dataset.vectorizer.load_pretrained_embed("../data/crawl-300d-2M.vec")
embedding = embedding.to(args.device)
cnn_classifier.embedding = nn.Embedding.from_pretrained (
    embedding,
    freeze=args.freeze_embedding
)
loss, acc, f1 = cnn_classifier.eval_net(mode='test')
print(loss)
print(acc)
print(f1)

### LSTM

Use a Long Short-Term Memory network for the previous configuration

In [ ]:
args.dataset = TextDataset.load_dataset_and_make_vectorizer("../data/tweets_products.csv", vectorizer_mode="padded")
args.pretrained_embedding = args.dataset.vectorizer.load_pretrained_embed("../data/crawl-300d-2M.vec")
args.name = "LSTM-Classifier.pth"

In [ ]:
lstm_classifier = LSTMClassifier(args)
lstm_classifier.setup()
lstm_classifier.fit()

In [ ]:
lstm_classifier.plot_logs("Accuracy", ["Train Accuracy", "Validation Accuracy"])

In [ ]:
lstm_classifier.plot_logs("Loss", ["Train Loss", "Validation Loss"])

In [ ]:
lstm_classifier.plot_logs("F1", ["Train F1-Score", "Validation F1-Score"])

In [ ]:
lstm_classifier.load_state()
loss, acc, f1 = lstm_classifier.eval_net(mode='test')
print(loss)
print(acc)
print(f1)

In [ ]:
args.dataset = TextDataset.load_dataset_and_make_vectorizer("../data/IMDB-dataset.csv", vectorizer_mode="padded")
embedding = args.dataset.vectorizer.load_pretrained_embed("../data/crawl-300d-2M.vec")
embedding = embedding.to(args.device)
lstm_classifier.embedding = nn.Embedding.from_pretrained (
    embedding,
    freeze=args.freeze_embedding
)

In [ ]:
loss, acc, f1 = lstm_classifier.eval_net(mode='test')
print(loss)
print(acc)
print(f1)